In [3]:
# importing required librabries 
import pdfplumber
import pandas as pd
from openai import OpenAI
import json
import re

# Function to extract text from PDF
def pdf_to_llm_input(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        text = ''
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + '\n'
    return text

# table extraction in JSON file 
def llm_table_extraction(pdf_text):
    client = OpenAI(api_key="sk-wQdE6fFbCcghdHP2osyYdlQFqKVdf6Hup-YF0Rz7wtT3BlbkFJiRrH5cKGj5uy7gaZog2EbM_RMgs7iwsKVuEoapAaEA")
    
    prompt = f"""ANALYZE THIS PDF CONTENT AND RETURN ALL TABLES IN PROPER JSON FORMAT:
{pdf_text}

STRICT FORMATTING RULES:
1. Output MUST be valid JSON only
2. Use this exact structure:
{{
  "tables": [
    {{
      "title": "Table Title",
      "columns": ["Column1", "Column2"],
      "rows": [
        ["Row1Val1", "Row1Val2"],
        ["Row2Val1", "Row2Val2"]
      ]
    }}
  ]
}}
3. Preserve ALL original values exactly
4. Escape special characters properly
5. Ensure JSON is properly closed"""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a JSON formatting expert. Only return valid JSON."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    
    raw_output = response.choices[0].message.content
    
# Cleaning common LLM JSON artifacts
    json_str = re.sub(r'^```json\n?|\n```$', '', raw_output, flags=re.IGNORECASE)
    
    try:
        data = json.loads(json_str)
        if "tables" not in data:
            raise ValueError("Missing 'tables' key in JSON response")
        return data
    except (json.JSONDecodeError, ValueError) as e:
        print(f"JSON Error: {str(e)}")
        print("Raw LLM response:")
        print(raw_output)
        raise

#Converting LLM JSON output to pandas DataFrames

def llm_to_dataframes(llm_output):
    dfs = []
    for table in llm_output["tables"]:
        df = pd.DataFrame(table["rows"], columns=table["columns"])
        dfs.append(df)
        globals()[f'df{len(dfs)}'] = df
        df.to_csv(f"table_{len(dfs)}.csv", index=False)
    return dfs

# Execution flow
pdf_path = "Annual-Report-for-the-Financial-Year-2023-2024.pdf"

#Converting PDF to LLM-digestible text
pdf_text = pdf_to_llm_input(pdf_path)

# LLM for table recognition
llm_output = llm_table_extraction(pdf_text)

# Converting LLM output to DataFrames
dataframes = llm_to_dataframes(llm_output)

# printing a table head
print(f"Extracted {len(dataframes)} tables:")
for i, df in enumerate(dataframes):
    #print(f"\ndf{i+1} ({df.shape[1]} columns x {df.shape[0]} rows):")
    print(df.head())

Extracted 1 tables:
                                         Particulars FY 2023-24 (Standalone)  \
0                                       Total Income                  247.97   
1                                  Total Expenditure                  264.21   
2  Profit / (Loss) before Tax from Continuing Ope...                  -16.24   
3                                  Exceptional Items                       -   
4  Profit / (Loss) from Continuing Operations, be...                  -16.24   

  FY 2022-23 (Standalone) FY 2023-24 (Consolidated) FY 2022-23 (Consolidated)  
0                  257.92                    318.13                    337.72  
1                  221.21                    504.43                    376.54  
2                   36.70                   -186.30                    -38.82  
3                       -                   -252.93                         -  
4                   36.70                   -439.24                    -38.82  
